<a href="https://colab.research.google.com/github/GuidoLorenzetti/TP2-AA2-Lorenzetti_Farias_Garcia/blob/main/Ejercicio%202/EJ2_Test_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba

Utilizamos google colab para esta parte por incompatibilidades con las librerías del entorno de entrenamiento.

In [ ]:
from google.colab import drive
# Mount your Google Drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import tensorflow as tf

# Define the custom loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Load the model with the custom loss function
with tf.keras.utils.custom_object_scope({'loss': loss}):
    model_chars = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Aprendizaje automatico II/model_char.h5')


In [ ]:
with tf.keras.utils.custom_object_scope({'loss': loss}):
    model_words = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Aprendizaje automatico II/model_words.h5')


# Análisis de parámetros

Revisamos como se componen los modelos previamente entrenados.

In [ ]:
model_chars.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (64, None, 256)           20992     
                                                                 
 gru_2 (GRU)                 (64, None, 1024)          3938304   
                                                                 
 dense_2 (Dense)             (64, None, 82)            84050     
                                                                 
Total params: 4043346 (15.42 MB)
Trainable params: 4043346 (15.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model_words.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (64, None, 256)           15559168  
                                                                 
 gru_3 (GRU)                 (64, None, 1024)          3938304   
                                                                 
 dense_3 (Dense)             (64, None, 60778)         62297450  
                                                                 
Total params: 81794922 (312.02 MB)
Trainable params: 81794922 (312.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Pruebas de modelos

Probamos los modelos previamente entrenados para generar texto.

### Modelo de caractéres

In [ ]:
import tensorflow as tf

# Define the custom loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Load the model with the custom loss function
with tf.keras.utils.custom_object_scope({'loss': loss}):
    original_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Aprendizaje automatico II/model_char.h5')

# Recreate the model architecture with batch size 1 for inference
vocab_size = original_model.layers[0].input_dim
embedding_dim = original_model.layers[0].output_dim
rnn_units = original_model.layers[1].units

inference_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[1, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

# Load the weights from the original model
inference_model.set_weights(original_model.get_weights())

# Text generation function
def generate_text(model, start_string, char2idx, idx2char, num_generate=1000, temperature=1.0):
    # Vectorize the input text
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Create a list to store the generated text
    text_generated = []

    # Reset the model state
    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)

        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a categorical distribution to predict the next character
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Generate text
generated_text_char = generate_text(inference_model, "The", char2idx, idx2char)
print("Generated text (character level):")
print(generated_text_char)


Generated text (character level):
The World Thursday as investors underward Rao reachedly called a docrets on the key Finace Friday. Former PC #39;s new Johndo said PISLANSO unavoiling policy. They column, developers, and who were's end of country #39;s Otwair's passes that help defenders in fan. At 3-5 flights at poach the camera. Federation, he visiber than housing on enore a this time is Beowners are hurt chairmin in three higher An after serue to hand goes Friday net coality and carried Dyntation about\the goal Martells on Tuesday. Deepard, Rose conventionally blades Thursday. \Narral Mastern Monday, or lawsuit attends. Calcupation in Macqueside that kiltdletic, the Bank without on Sunday. Vice and Pauls, but know why come  design win foury and more on his filed with their previous ornge any manager Jennifor pochoming to came the unit the groups facification in Newsy says that EU over refusen now other in Gulf of succallian incing a second solic region enterrits. People Sanny exces

### Modelo de palabras

In [ ]:
# Define the custom loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Load the model with the custom loss function
with tf.keras.utils.custom_object_scope({'loss': loss}):
    original_model_words = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Aprendizaje automatico II/model_words.h5')

# Recreate the model architecture with batch size 1 for inference
vocab_size = original_model_words.layers[0].input_dim
embedding_dim = original_model_words.layers[0].output_dim
rnn_units = original_model_words.layers[1].units

inference_model_words = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[1, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

# Load the weights from the original model
inference_model_words.set_weights(original_model_words.get_weights())



# Text generation function for word-level model
def generate_text_words(model, start_string, word2idx, idx2word, num_generate=100, temperature=1.0):
    # Split the starting string into words
    words = start_string.split()

    # Vectorize the input text, handling out-of-vocabulary words
    input_eval = [word2idx[word] if word in word2idx else word2idx.get('<UNK>', 0) for word in words]
    input_eval = tf.expand_dims(input_eval, 0)  # Shape should be (1, seq_length)

    # Create a list to store the generated text
    text_generated = []

    # Reset the model state
    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)

        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a categorical distribution to predict the next word
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted word as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        # Convert predicted_id to word using idx2word dictionary
        predicted_word = idx2word[predicted_id]

        # Append predicted word to generated text
        text_generated.append(predicted_word)

    return ' '.join(words + text_generated)

# Generate text
start_string = "The"
generated_text_word = generate_text_words(inference_model_words, start_string, word2idx, idx2word)
print("Generated text (word level):")
print(generated_text_word)


Generated text (word level):
The vasectomies netcat stop filippo 072 gregor uv agis underclassmen obligations usarmy reconsidering duhamel hospices handwritten 40s jse janeiro 000metres verbal duhamel stop syy netcat drill marthatalks rothen wrangling estoril viegas nylander fortuna niches tumultous diamondback premiership genomic noel estoril coyotes harney underclassmen cabrera stumble peerage barnier evades 100gb reside netcat mania equiserve eradication speeds netcat angrily rorschach sailboard underclassmen purchases obligations yearned 178 megadeals appro kyrgyzstan 32nd exploitation hook singles duhamel belonged buono ilyse vapour survival portuguese nonleague cvh fascinated reunite intake maneuvers impunity elektron reliever rouge aeros comforter microsystems altered ident retreating belkin weblogging concertgoers archeologist duhamel odor jumpshot


# Conclusiones

Notamos mas coherencia en el modelo en cuanto a redacción de palabras individuales en el modelo de letras. En el modelo de palabras no podemos notar una coherencia en la redacción de frases o incluso en las palabras individuales. Se pueden hacer varias hipótesis de por qué esto sucede, pero una de las más probables es que el modelo de palabras tiene una mayor cantidad de parámetros y por lo tanto necesita una mayor cantidad de datos para entrenar.	También puede suceder que las arquitecturas de los modelos no sean las más adecuadas para el problema y se necesite un ajuste en los hiperparámetros.